# Lending Club Statistics 를 선형 회귀 분석

이 데이터셋에는 현재 대출 상태 (대출중, 연체, 갚음 등) 및 최신 지불 정보를 포함하여 명시된 기간 동안 발행 된 모든 대출에 대한 완전한 대출 데이터가 들어 있습니다. "현재"를 통한 대출 데이터가 포함 된 파일에는 이전에 완료 한 달력 분기를 통해 발행 된 모든 대출에 대한 전체 대출 데이터가 들어 있습니다.

데이터셋 URL : https://www.lendingclub.com/info/download-data.action

* 데이터 셋
    * 

In [6]:
# Load Packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
#import dask.dataframe as dd
#from dask.diagnostics import ProgressBar

# Settings
%matplotlib inline
sns.set(font="NanumGothic")
pbar = ProgressBar()
pbar.register()

|ddd|aa|ww|tt|
|---|---|---|---|


In [1]:
dataQ1 = pd.read_csv("/home/dockeruser/hosthome/Documents/workspaces/projects/fc_team_project2/datasets/LoanStats_2016Q1.csv", skiprows=1)
dataQ2 = pd.read_csv("/home/dockeruser/hosthome/Documents/workspaces/projects/fc_team_project2/datasets/LoanStats_2016Q2.csv", skiprows=1)
dataQ3 = pd.read_csv("/home/dockeruser/hosthome/Documents/workspaces/projects/fc_team_project2/datasets/LoanStats_2016Q3.csv", skiprows=1)
dataQ4 = pd.read_csv("/home/dockeruser/hosthome/Documents/workspaces/projects/fc_team_project2/datasets/LoanStats_2016Q4.csv", skiprows=1)

/home/dockeruser/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/dockeruser/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
data = data.dropna(axis=0)

In [52]:
data.columns

Index([u'AmountRequested', u'AmountFundedByInvestors', u'InterestRate',
       u'LoanLength', u'LoanPurpose', u'DebtToIncomeRatio', u'State',
       u'HomeOwnership', u'MonthlyIncome', u'FICORange', u'OpenCREDITLines',
       u'RevolvingCREDITBalance', u'InquiriesintheLast6Months',
       u'EmploymentLength'],
      dtype='object')

In [60]:
asdf = sm.OLS.from_formula("InterestRate ~ MonthlyIncome", data=data).fit()

In [61]:
asdf.summary()

ValueError: shapes (2498,273) and (2498,273) not aligned: 273 (dim 1) != 2498 (dim 0)

In [59]:
data.head()

,AmountRequested,AmountFundedByInvestors,InterestRate,LoanLength,LoanPurpose,DebtToIncomeRatio,State,HomeOwnership,MonthlyIncome,FICORange,OpenCREDITLines,RevolvingCREDITBalance,InquiriesintheLast6Months,EmploymentLength
81174,20000,20000.0,8.90%,36 months,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,735-739,14.0,14272.0,2.0,< 1 year
99592,19200,19200.0,12.12%,36 months,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,715-719,12.0,11140.0,1.0,2 years
80059,35000,35000.0,21.98%,60 months,debt_consolidation,23.81%,CA,MORTGAGE,11500.00,690-694,14.0,21977.0,1.0,2 years
15825,10000,9975.0,9.99%,36 months,debt_consolidation,14.30%,KS,MORTGAGE,3833.33,695-699,10.0,9346.0,0.0,5 years
33182,12000,12000.0,11.71%,36 months,credit_card,18.78%,NJ,RENT,3195.00,695-699,11.0,14469.0,0.0,9 years


In [56]:
data.count()

AmountRequested              2498
AmountFundedByInvestors      2498
InterestRate                 2498
LoanLength                   2498
LoanPurpose                  2498
DebtToIncomeRatio            2498
State                        2498
HomeOwnership                2498
MonthlyIncome                2498
FICORange                    2498
OpenCREDITLines              2498
RevolvingCREDITBalance       2498
InquiriesintheLast6Months    2498
EmploymentLength             2498
dtype: int64